<a href="https://colab.research.google.com/github/hakurennnn/cnn/blob/master/moved%20to%20colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import os

In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import glob
import imageio
from keras.applications import densenet
from tensorflow.keras import layers

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:

## LOCAL RUN PATHS
main_path = '/content/drive/MyDrive' #palitan ng main folder

test_path = os.path.join(main_path,"test")
train_path = os.path.join(main_path,"train")

train_normal = glob.glob(train_path+"/NORMAL/*.jpeg")
train_pneumonia = glob.glob(train_path+"/PNEUMONIA/*.jpeg")

test_normal = glob.glob(test_path+"/NORMAL/*.jpeg")
test_pneumonia = glob.glob(test_path+"/PNEUMONIA/*.jpeg")



In [5]:
# combine train_normal and train_pneumonia
train_list = train_normal + train_pneumonia
test_list = test_normal + test_pneumonia

df_train = pd.DataFrame({'class': ['Normal']*len(train_normal) + ['Pneumonia']*len(train_pneumonia),
                         'image': train_list})

df_test = pd.DataFrame({'class': ['Normal']*len(test_normal) + ['Pneumonia']*len(test_pneumonia),
                        'image': test_list})


In [6]:
class_count = df_test['class'].value_counts()
print(class_count)

Pneumonia    390
Normal       234
Name: class, dtype: int64


In [7]:
print(df_train)

          class                                              image
0        Normal  /content/drive/MyDrive/train/NORMAL/NORMAL-324...
1        Normal  /content/drive/MyDrive/train/NORMAL/NORMAL-337...
2        Normal  /content/drive/MyDrive/train/NORMAL/NORMAL-327...
3        Normal  /content/drive/MyDrive/train/NORMAL/NORMAL-337...
4        Normal  /content/drive/MyDrive/train/NORMAL/NORMAL-352...
...         ...                                                ...
5227  Pneumonia  /content/drive/MyDrive/train/PNEUMONIA/BACTERI...
5228  Pneumonia  /content/drive/MyDrive/train/PNEUMONIA/BACTERI...
5229  Pneumonia  /content/drive/MyDrive/train/PNEUMONIA/BACTERI...
5230  Pneumonia  /content/drive/MyDrive/train/PNEUMONIA/BACTERI...
5231  Pneumonia  /content/drive/MyDrive/train/PNEUMONIA/BACTERI...

[5232 rows x 2 columns]


In [8]:
print(df_test)

         class                                              image
0       Normal  /content/drive/MyDrive/test/NORMAL/NORMAL-1141...
1       Normal  /content/drive/MyDrive/test/NORMAL/NORMAL-1110...
2       Normal  /content/drive/MyDrive/test/NORMAL/NORMAL-1160...
3       Normal  /content/drive/MyDrive/test/NORMAL/NORMAL-1152...
4       Normal  /content/drive/MyDrive/test/NORMAL/NORMAL-1212...
..         ...                                                ...
619  Pneumonia  /content/drive/MyDrive/test/PNEUMONIA/VIRUS-84...
620  Pneumonia  /content/drive/MyDrive/test/PNEUMONIA/VIRUS-90...
621  Pneumonia  /content/drive/MyDrive/test/PNEUMONIA/VIRUS-95...
622  Pneumonia  /content/drive/MyDrive/test/PNEUMONIA/VIRUS-95...
623  Pneumonia  /content/drive/MyDrive/test/PNEUMONIA/VIRUS-90...

[624 rows x 2 columns]


# Creating the model

In [9]:
# Create validation set
train_df, val_df = train_test_split(df_train, test_size = 0.20, random_state = 32, stratify = df_train['class'])

In [10]:
batch = 32
img_size = 224

In [11]:
train_gen = ImageDataGenerator(rescale = 1./255,
                               shear_range = 0.2,
                               zoom_range = 0.2,
                               vertical_flip = True)

val_gen = ImageDataGenerator(rescale=1/255)

ds_train = train_gen.flow_from_dataframe(train_df,
                                             x_col = 'image',
                                             y_col = 'class',
                                             target_size = (img_size, img_size),
                                             batch_size = batch,
                                             class_mode = 'binary',
                                             color_mode = 'grayscale'
                                             )

ds_val = val_gen.flow_from_dataframe(val_df,
                                            #directory=train_path,
                                            x_col = 'image',
                                            y_col = 'class',
                                            target_size = (img_size, img_size),
                                            batch_size = batch,
                                            class_mode = 'binary',
                                            color_mode = 'grayscale')

ds_test = val_gen.flow_from_dataframe(df_test,
                                       x_col='image',
                                       y_col='class',
                                       target_size=(img_size, img_size),
                                       batch_size=batch,
                                       class_mode='binary',
                                       color_mode='grayscale')

Found 4185 validated image filenames belonging to 2 classes.
Found 1047 validated image filenames belonging to 2 classes.
Found 624 validated image filenames belonging to 2 classes.


In [12]:
# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='auto', restore_best_weights=True)

In [13]:
def predict_image(model, img_path, preprocess_input_fn, decode_predictions_fn, target_size=(img_size,img_size)):

    img = tensorflow.keras.utils.load_img(img_path, target_size=target_size)
    x = tensorflow.keras.utils.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input_fn(x)
    
    preds = model.predict(x)
    predictions_df = pd.DataFrame(decode_predictions_fn(preds, top=10)[0])
    predictions_df.columns = ["Predicted Class", "Name", "Probability"]
    return predictions_df

In [14]:
pre_model = densenet.DenseNet121(weights=None,
                                 include_top=False,
                                 input_shape=(img_size, img_size, 1),
                                 pooling='avg'
                                )

In [15]:
model = Sequential()

# Add DenseNet121 base
model.add(pre_model)

# Add new output layer with 2 units and "softmax" activation
model.add(Dense(1, activation='sigmoid'))

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet121 (Functional)    (None, 1024)              7031232   
                                                                 
 dense (Dense)               (None, 1)                 1025      
                                                                 
Total params: 7,032,257
Trainable params: 6,948,609
Non-trainable params: 83,648
_________________________________________________________________


In [17]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [19]:
results = model.fit(ds_train,
                        batch_size=batch,
                        epochs=10,
                        validation_data=ds_val,
                        steps_per_epoch=(len(train_df)/batch),
                        callbacks=[early_stopping], 
                        verbose=1)


Epoch 1/10
130/130 [==============================] - 632s 5s/step - loss: 0.2337 - accuracy: 0.9035 - val_loss: 3.8740 - val_accuracy: 0.7421
Epoch 2/10
130/130 [==============================] - 90s 689ms/step - loss: 0.1872 - accuracy: 0.9264 - val_loss: 6.0383 - val_accuracy: 0.7421
Epoch 3/10
130/130 [==============================] - 97s 740ms/step - loss: 0.1842 - accuracy: 0.9254 - val_loss: 2.3706 - val_accuracy: 0.7421
Epoch 4/10
130/130 [==============================] - 89s 678ms/step - loss: 0.1673 - accuracy: 0.9355 - val_loss: 0.6758 - val_accuracy: 0.7421
Epoch 5/10
130/130 [==============================] - 90s 691ms/step - loss: 0.1648 - accuracy: 0.9386 - val_loss: 1.4067 - val_accuracy: 0.7421
Epoch 6/10
130/130 [==============================] - 89s 682ms/step - loss: 0.1567 - accuracy: 0.9443 - val_loss: 1.1110 - val_accuracy: 0.7421
Epoch 7/10
130/130 [==============================] - 90s 688ms/step - loss: 0.1419 - accuracy: 0.9443 - val_loss: 7.3577 - val_accu